In [7]:
#Imports
import numpy as np
import pandas as pd
import sys, os, random
import importlib
from sklearn.preprocessing import StandardScaler

#Class Import
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../.."))) #Allow for imports from src
from src.models import ML_Class_2
importlib.reload(ML_Class_2) #Ensures file is uptodate!
from src.models.ML_Class_2 import Model_Tester_V2

#Utils Import
from src.models.model_specs import MODEL_SPECS
from src.models.perf_utils import track_performance

#Set Seed
os.environ["PYTHONHASHSEED"] = "1945"
random.seed(1945)
np.random.seed(1945)

### Models Tested:

| Key | Algorithm | Library |
|:----|:-----------|:---------|
| **dt** | Decision Tree Classifier | scikit-learn |
| **rf** | Random Forest Classifier | scikit-learn |
| **et** | Extra Trees Classifier | scikit-learn |
| **bag** | Bagging Classifier (Tree Base) | scikit-learn |
| **gb** | Gradient Boosting Classifier | scikit-learn |
| **ada** | AdaBoost Classifier | scikit-learn |
| **qda** | Quadratic Discriminant Analysis | scikit-learn |
| **xgb** | XGBoost Classifier | xgboost |
| **lgbm** | LightGBM Classifier | lightgbm |
| **cat** | CatBoost Classifier | catboost |

 **Note:**  
Preliminary algorithm tests were done in Algorithm_Test_2. In this notebook, further optimization and comparison are done!

In [8]:
# --- Data Loading and Preprocessing --- 

#Complied data of convoys
#Routes examined are HX, SC, OB, ON, ONS
df = pd.read_csv('/Users/matthewplambeck/Desktop/Convoy Predictor/data/processed/Complete_Convoy_Data.csv')
df = df.drop(columns=['Unnamed: 0'])
df.shape #Test
#Drop unecessary/redundent features
df = df.drop(columns=['Convoy Number', 'Number of Ships Sunk', 'Depart_Date', 'Arrival/Dispersal Date', 'Number of Escorts Sunk', \
                         'Number of Stragglers Sunk', 'Total Tons of Ships Sunk', 'Escort Sink Percentage', 'Straggler Sink Percentage'])
df.reset_index(drop=True).head(3)
#Feature Names for later feature analysis:
feature_names = list(df)
feature_names[:-1] #Drop Risk (y)
#Convert Overall Sink Percentage to binary 1( High)
df['Risk'] = (df['Overall Sink Percentage'] > 0).astype(int) 
#Risk is binary based off whether a ship was sunk while in a convoy:  (0 = No Ships Sunk, 1 = At Least One Ship Sunk)
X = np.array(df.drop(columns=['Overall Sink Percentage', 'Risk'])) #Remove Overall Sink Percentage as it leaks data
y = df['Risk'].values #Prediction value

**Start of Algorithms Tests**

In [9]:
#Decision Tree

spec = MODEL_SPECS["dt"]
dt = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

dt.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("dt_optimize")
def run_dt_opt():
    dt.optimize(scoring="recall")

run_dt_opt()
dt_results = dt.evaluate(show_plots=False)


Best Hyperparameters Found:
{'class_weight': 'balanced', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2}
Best Cross-Validation Recall: 0.7706

Performance Stats:
dt_optimize completed in 5.85s | ΔRSS 2.12 MB | CPU 67.8%

Applied decision threshold: 0.8491

DecisionTreeClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.92      0.89       185
           1       0.59      0.40      0.48        50

    accuracy                           0.81       235
   macro avg       0.72      0.66      0.68       235
weighted avg       0.79      0.81      0.80       235


ROC AUC Score: 0.7241
Matthews Correlation Coefficient (MCC): 0.3773
Balanced Accuracy: 0.6622
DecisionTreeClassifier Confusion Matrix:
[[171  14]
 [ 30  20]]


In [10]:
#Random Forest

spec = MODEL_SPECS["rf"]
rf = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

rf.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("rf_optimize")
def run_rf_opt():
    rf.optimize(scoring="recall")

run_rf_opt()
rf_results = rf.evaluate(show_plots=False)


Best Hyperparameters Found:
{'class_weight': 'balanced', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Best Cross-Validation Recall: 0.5673

Performance Stats:
rf_optimize completed in 39.70s | ΔRSS 3.69 MB | CPU 79.4%

Applied decision threshold: 0.4961

RandomForestClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       185
           1       0.59      0.64      0.62        50

    accuracy                           0.83       235
   macro avg       0.75      0.76      0.75       235
weighted avg       0.84      0.83      0.83       235


ROC AUC Score: 0.8251
Matthews Correlation Coefficient (MCC): 0.5069
Balanced Accuracy: 0.7605
RandomForestClassifier Confusion Matrix:
[[163  22]
 [ 18  32]]


In [11]:
#Extra Trees

spec = MODEL_SPECS["et"]
et = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

et.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("et_optimize")
def run_et_opt():
    et.optimize(scoring="recall")

run_et_opt()
et_results = et.evaluate(show_plots=False)


Best Hyperparameters Found:
{'class_weight': 'balanced', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'n_estimators': 400}
Best Cross-Validation Recall: 0.7462

Performance Stats:
et_optimize completed in 14.38s | ΔRSS -10.50 MB | CPU 82.7%

Applied decision threshold: 0.5914

ExtraTreesClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       185
           1       0.62      0.58      0.60        50

    accuracy                           0.83       235
   macro avg       0.75      0.74      0.75       235
weighted avg       0.83      0.83      0.83       235


ROC AUC Score: 0.8132
Matthews Correlation Coefficient (MCC): 0.4939
Balanced Accuracy: 0.7414
ExtraTreesClassifier Confusion Matrix:
[[167  18]
 [ 21  29]]


In [12]:
#Bagging Classifier 

spec = MODEL_SPECS["bag"]
bag = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

bag.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("bag_optimize")
def run_bag_opt():
    bag.optimize(scoring="recall")

run_bag_opt()
bag_results = bag.evaluate(show_plots=False)


Best Hyperparameters Found:
{'bootstrap': True, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 150}
Best Cross-Validation Recall: 0.3879

Performance Stats:
bag_optimize completed in 8.60s | ΔRSS 3.28 MB | CPU 77.0%

Applied decision threshold: 0.5467

BaggingClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.97      0.91       185
           1       0.79      0.38      0.51        50

    accuracy                           0.85       235
   macro avg       0.82      0.68      0.71       235
weighted avg       0.84      0.85      0.82       235


ROC AUC Score: 0.8251
Matthews Correlation Coefficient (MCC): 0.4771
Balanced Accuracy: 0.6765
BaggingClassifier Confusion Matrix:
[[180   5]
 [ 31  19]]


In [13]:
#Gradient Boosting Classifier 

spec = MODEL_SPECS["gb"]
gb = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

gb.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("gb_optimize")
def run_gb_opt():
    gb.optimize(scoring="recall")

run_gb_opt()
gb_results = gb.evaluate(show_plots=False)




Best Hyperparameters Found:
{'learning_rate': 0.07, 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 600, 'subsample': 1.0}
Best Cross-Validation Recall: 0.4432

Performance Stats:
gb_optimize completed in 57.23s | ΔRSS -16.75 MB | CPU 86.4%

Applied decision threshold: 0.7785

GradientBoostingClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.97      0.89       185
           1       0.68      0.26      0.38        50

    accuracy                           0.82       235
   macro avg       0.76      0.61      0.63       235
weighted avg       0.80      0.82      0.78       235


ROC AUC Score: 0.8001
Matthews Correlation Coefficient (MCC): 0.3416
Balanced Accuracy: 0.6138
GradientBoostingClassifier Confusion Matrix:
[[179   6]
 [ 37  13]]


In [14]:
#AdaBoost

spec = MODEL_SPECS["ada"]
ada = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

ada.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("ada_optimize")
def run_ada_opt():
    ada.optimize(scoring="recall")

run_ada_opt()
ada_results = ada.evaluate(show_plots=False)


Best Hyperparameters Found:
{'learning_rate': 0.2, 'n_estimators': 400}
Best Cross-Validation Recall: 0.2039

Performance Stats:
ada_optimize completed in 3.10s | ΔRSS 1.55 MB | CPU 62.3%

Applied decision threshold: 0.4200

AdaBoostClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       185
           1       0.60      0.56      0.58        50

    accuracy                           0.83       235
   macro avg       0.74      0.73      0.73       235
weighted avg       0.82      0.83      0.82       235


ROC AUC Score: 0.7825
Matthews Correlation Coefficient (MCC): 0.4679
Balanced Accuracy: 0.7286
AdaBoostClassifier Confusion Matrix:
[[166  19]
 [ 22  28]]


In [16]:
#QuadraticDiscriminantAnalysis

spec = MODEL_SPECS["qda"]
qda = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    scaler=StandardScaler(),
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

qda.train_test_split(X, y, train_size=0.8, random_state=1945)

@track_performance("qda_optimize")
def run_qda_opt():
    qda.optimize(scoring="recall")

run_qda_opt()
qda_results = qda.evaluate(show_plots=False)


Best Hyperparameters Found:
{'model__reg_param': 0.0}
Best Cross-Validation Recall: 0.6022

Performance Stats:
qda_optimize completed in 0.13s | ΔRSS 13.50 MB | CPU 28.7%

Applied decision threshold: 0.4781

QuadraticDiscriminantAnalysis Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.79      0.85       185
           1       0.48      0.70      0.57        50

    accuracy                           0.77       235
   macro avg       0.69      0.75      0.71       235
weighted avg       0.82      0.77      0.79       235


ROC AUC Score: 0.7897
Matthews Correlation Coefficient (MCC): 0.4374
Balanced Accuracy: 0.7473
QuadraticDiscriminantAnalysis Confusion Matrix:
[[147  38]
 [ 15  35]]


In [20]:
#XGBoost

spec = MODEL_SPECS["xgb"]
xgb = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

xgb.train_test_split(X, y, train_size=0.8, random_state=1945)
if callable(xgb.parameter_grid):
    xgb.parameter_grid = xgb.parameter_grid(xgb.y_train)
@track_performance("xgb_optimize")
def run_xgb_opt():
    xgb.optimize(scoring="recall")

run_xgb_opt()
xgb_results = xgb.evaluate(show_plots=False)


Best Hyperparameters Found:
{'colsample_bytree': 0.9, 'learning_rate': 0.04, 'max_depth': 3, 'n_estimators': 600, 'scale_pos_weight': 5.507462686567164, 'subsample': 0.75}
Best Cross-Validation Recall: 0.5973

Performance Stats:
xgb_optimize completed in 30.28s | ΔRSS -37.97 MB | CPU 58.1%

Applied decision threshold: 0.6873

XGBClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.92      0.89       185
           1       0.58      0.42      0.49        50

    accuracy                           0.81       235
   macro avg       0.72      0.67      0.69       235
weighted avg       0.80      0.81      0.80       235


ROC AUC Score: 0.7788
Matthews Correlation Coefficient (MCC): 0.3851
Balanced Accuracy: 0.6695
XGBClassifier Confusion Matrix:
[[170  15]
 [ 29  21]]


In [ ]:
#LightGBM

spec = MODEL_SPECS["lgbm"]
lgbm = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

lgbm.train_test_split(X, y, train_size=0.8, random_state=1945)
if callable(lgbm.parameter_grid):
    lgbm.parameter_grid = lgbm.parameter_grid(lgbm.y_train)
    
@track_performance("lgbm_optimize")
def run_lgbm_opt():
    lgbm.optimize(scoring="recall")

run_lgbm_opt()
lgbm_results = lgbm.evaluate(show_plots=False)

In [23]:
lgbm.evaluate(show_plots=False)

Applied decision threshold: 0.9958

LGBMClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       185
           1       0.78      0.14      0.24        50

    accuracy                           0.81       235
   macro avg       0.79      0.56      0.56       235
weighted avg       0.80      0.81      0.75       235


ROC AUC Score: 0.7730
Matthews Correlation Coefficient (MCC): 0.2755
Balanced Accuracy: 0.5646
LGBMClassifier Confusion Matrix:
[[183   2]
 [ 43   7]]


/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


{'model_name': 'LGBMClassifier',
 'classification_report': {'0': {'precision': 0.8097345132743363,
   'recall': 0.9891891891891892,
   'f1-score': 0.8905109489051095,
   'support': 185.0},
  '1': {'precision': 0.7777777777777778,
   'recall': 0.14,
   'f1-score': 0.23728813559322035,
   'support': 50.0},
  'accuracy': 0.8085106382978723,
  'macro avg': {'precision': 0.7937561455260571,
   'recall': 0.5645945945945946,
   'f1-score': 0.5638995422491649,
   'support': 235.0},
  'weighted avg': {'precision': 0.8029352078495366,
   'recall': 0.8085106382978723,
   'f1-score': 0.7515273716047075,
   'support': 235.0}},
 'confusion_matrix': array([[183,   2],
        [ 43,   7]]),
 'roc_auc': 0.7729729729729728,
 'mcc': 0.2754999755348547,
 'balanced_accuracy': 0.5645945945945946,
 'notes': 'Enable early stopping via fit_with_hooks; pass valid_sets with (X_val,y_val).'}

In [21]:
#CatBoost

spec = MODEL_SPECS["cat"]
cat = Model_Tester_V2(
    model=spec["estimator"],
    parameter_grid=spec["grid_small"],
    cv_folds=5,
    feature_names=feature_names,
    model_config=spec["config"],)

cat.train_test_split(X, y, train_size=0.8, random_state=1945)
if callable(cat.parameter_grid):
    cat.parameter_grid = cat.parameter_grid(cat.y_train)
@track_performance("cat_optimize")
def run_cat_opt():
    cat.optimize(scoring="recall")

run_cat_opt()
cat_results = cat.evaluate(show_plots=False)


Best Hyperparameters Found:
{'depth': 4, 'iterations': 1200, 'l2_leaf_reg': 4.0, 'learning_rate': 0.04, 'scale_pos_weight': 5.507462686567164}
Best Cross-Validation Recall: 0.5227

Performance Stats:
cat_optimize completed in 58.19s | ΔRSS 34.50 MB | CPU 62.1%

Applied decision threshold: 0.3898

CatBoostClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       185
           1       0.57      0.42      0.48        50

    accuracy                           0.81       235
   macro avg       0.71      0.67      0.68       235
weighted avg       0.79      0.81      0.80       235


ROC AUC Score: 0.7717
Matthews Correlation Coefficient (MCC): 0.3748
Balanced Accuracy: 0.6668
CatBoostClassifier Confusion Matrix:
[[169  16]
 [ 29  21]]
